# Hyperparameter optimization for PPO RL agent using ASHA scheduler and Random search

In this lesson, we learn to use the ASHA scheduler to stop less-promising trials (with bad hyperparameter value combinations) and speeding up the training process while optimizing the hyperparameters.

In [1]:
from pathlib import Path
from ray import air, tune
from ray.tune.schedulers import ASHAScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm

For this notebook, we will try to optimize the learning rate `lr` and the discount factor `gamma` hyperparameters (same from [lesson 2 notebook 2](2-optimize_ppo_hyperparameters_cartpole.ipynb)).

In [2]:
search_space = {
    "lr": tune.loguniform(1e-5, 1),
    "gamma": tune.choice(
        [
            0.5,
            0.6,
            0.7,
            0.8,
            0.9,
            0.95,
            0.98,
            0.99,
            0.995,
            0.999,
            0.9999,
        ]
    ),
}

Similarly to [lesson 2 notebook 2](2-optimize_ppo_hyperparameters_cartpole.ipynb), we use a random search algorithm.

In [3]:
search_algo = tune.search.basic_variant.BasicVariantGenerator()  # Random search

In this example, we utilize an ASHA scheduler to stop unpromising trials (with bad performance).

In [4]:
scheduler_algo = ASHAScheduler(
    time_attr="training_iteration",  # Metric to use for time comparison
    max_t=10,  # Max time units per trial.
    grace_period=1,  # Only stop trials at least this old in time.
    reduction_factor=3,  # Used to set halving rate and amount.
    brackets=1,  # Number of brackets. Each bracket has a different halving rate, specified by the reduction factor.
)  # ASHA trial scheduler

Once the search and scheduler algorithms are defined, we can define our Tune configuration:

In [5]:
number_trials = 10
tune_config = tune.TuneConfig(
    metric="env_runners/episode_reward_mean",  # That's the metric we want to maximize/minimize
    mode="max",  # Here we indicate we want to maximize the metric env_runners/episode_reward_mean
    scheduler=scheduler_algo,
    search_alg=search_algo,
    num_samples=number_trials,  # Number of trials to run
)

Now, it's time to train our PPO RL agent using this Tune configurations. When executing the cell below, pay attention to the number of trials with status PENDING and RUNNING.

In [6]:
config = PPOConfig().environment("CartPole-v1")
stop = {
    "training_iteration": 10,
}
checkpoint_frequency = 0
store_results_path = str(Path("./ray_results/").resolve()) + "/nb_3/"
agent_name = "ppo_cartpole"

tuner = tune.Tuner(
    "PPO",
    param_space={
        **config.to_dict(),
        **search_space,
    },  # Here we mix the Algo config with the search space
    tune_config=tune_config,
    run_config=air.RunConfig(
        storage_path=store_results_path,
        name=agent_name,
        stop=stop,
        verbose=2,
        checkpoint_config=air.CheckpointConfig(
            checkpoint_frequency=checkpoint_frequency,
            checkpoint_at_end=True,
        ),
    ),
)
results = tuner.fit()
print(results)

2024-12-01 00:53:44,932	INFO worker.py:1783 -- Started a local Ray instance.
2024-12-01 00:53:45,446	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-12-01 00:53:45,448	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, act

(PPO pid=574831) Install gputil for GPU system monitoring.
(PPO pid=574831) 2024-12-01 00:53:55,502	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


Trial name,agent_timesteps_total,counters,custom_metrics,episode_media,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_CartPole-v1_b6f11_00000,4000,"{'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}",{},{},"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(1.8215415), 'cur_kl_coeff': np.float64(0.20000000000000004), 'cur_lr': np.float64(1.3387362386843284e-05), 'total_loss': np.float64(9.208352335037723), 'policy_loss': np.float64(-0.03128230159711694), 'vf_loss': np.float64(9.235793708473123), 'vf_explained_var': np.float64(-0.000993846244709466), 'kl': np.float64(0.019204470895124115), 'entropy': np.float64(0.6744765489332137), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(128.0), 'num_grad_updates_lifetime': np.float64(465.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(464.5)}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}",4000,4000,4000,4000,4000,4000,414.267,4000,4000,414.267,2,0,0,4000,"{'cpu_util_percent': np.float64(36.84285714285714), 'ram_util_percent': np.float64(38.471428571428575)}","{'training_iteration_time_ms': 9655.61, 'restore_workers_time_ms': 0.021, 'training_step_time_ms': 9655.557, 'sample_time_ms': 2065.697, 'load_time_ms': 0.488, 'load_throughput': 8204017.604, 'learn_time_ms': 7584.957, 'learn_throughput': 527.36, 'synch_weights_time_ms': 3.752}"
PPO_CartPole-v1_b6f11_00001,12000,"{'num_env_steps_sampled': 12000, 'num_env_steps_trained': 12000, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 12000}",{},{},"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(0.4670086), 'cur_kl_coeff': np.float64(0.3), 'cur_lr': np.float64(0.00014789929860875513), 'total_loss': np.float64(9.807838360981275), 'policy_loss': np.float64(-0.022226133475941354), 'vf_loss': np.float64(9.82706548219086), 'vf_explained_var': np.float64(0.006808120012283325), 'kl': np.float64(0.009996683280784069), 'entropy': np.float64(0.5744201308937483), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(128.0), 'num_grad_updates_lifetime': np.float64(2325.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(464.5)}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 12000, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 12000}",12000,12000,12000,12000,12000,4000,415.999,12000,4000,415.999,2,0,0,4000,"{'cpu_util_percent': np.float64(38.94285714285714), 'ram_util_percent': np.float64(38.07142857142857)}","{'training_iteration_time_ms': 9722.392, 'restore_workers_time_ms': 0.019, 'training_step_time_ms': 9722.342, 'sample_time_ms': 2114.384, 'load_time_ms': 0.35, 'load_throughput': 11428621.253, 'learn_time_ms': 7602.946, 'learn_throughput': 526.112, 'synch_weights_time_ms': 4.072}"
PPO_CartPole-v1_b6f11_00002,4000,"{'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}",{},{},"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(5.515938), 'cur_kl_coeff': np.float64(0.20000000000000004), 'cur_lr': np.float64(0.0034043997621444467), 'total_loss': np.float64(5.423406871159871), 'policy_loss': np.float64(-0.05221704922175856), 'vf_loss': np.float64(5.468204048884812), 'vf_explained_var': np.flo

(PPO pid=574827) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/lasse/ray_minicourse/lesson_2/ray_results/nb_3/ppo_cartpole/PPO_CartPole-v1_b6f11_00000_0_gamma=0.9999,lr=0.0000_2024-12-01_00-53-45/checkpoint_000000)
(PPO pid=574829) Install gputil for GPU system monitoring. [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(PPO pid=574829) 2024-12-01 00:53:55,724	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future! [repeated 4x across cluster]
(PPO pid=574829) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/lasse/ray_minicourse/lesson_2/ray_results/nb_3/ppo_cartpole/PPO_CartPole-v1_b6f11_00002_2_gamma=0.9950,lr=0.0034_2024-12-01_

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(1.8215415), 'cur_kl_coeff': np.float64(0.20000000000000004), 'cur_lr': np.float64(1.3387362386843284e-05), 'total_loss': np.float64(9.208352335037723), 'policy_loss': np.float64(-0.03128230159711694), 'vf_loss': np.float64(9.235793708473123), 'vf_explained_var': np.float64(-0.000993846244709466), 'kl': np.float64(0.019204470895124115), 'entropy': np.float64(0.6744765489332137), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(128.0), 'num_grad_updates_lifetime': np.float64(465.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(464.5)}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'env_runners': {'episode_reward_max': 74.0, 'episode_reward_mi

Pay attention to the `Trial progress` in the logs above in the column `agent_timesteps_total`. Not all the trials have completed the $40000$ pre-defined steps (training iterations * batch size = 10 * 4000). The ASHA scheduler finalized the unpromised trials before finishing all the training iterations to save time and speedup the hyperparameter optimization process.

Now, let's observe the reward progress during the training in tensorboard looking at the `episode_reward_mean` metric.

In [7]:
%load_ext tensorboard
%tensorboard --logdir ray_results/nb_3

The training reward shows that not all the trials completed the 10 iterations. This is because the ASHA scheduler stops the trials that are not performing well and only the best trials finished the training.